# IFS Software walkthrough

Use the software to generate monochromatic images for wavelength calibration (optional)

In [1]:
import tools
import glob
from params import Params
import numpy as np
import logging as log
from tools.initLogger import initLogger



Load the parameter class defining the IFS, params.py. All parameters can be changed here. Note there might be combinations of parameters which might be inconsistent. At the same time as loading the parameters, start a logger.

In [2]:
par = Params()
initLogger(par.exportDir+'/IFS.log')
par.R = 110

In [3]:
lamlist = np.arange(605,725+1,10)



In [4]:
# from tools.wavecal import createWavecalFiles
# createWavecalFiles(par,lamlist=lamlist)

## Compute wavelength calibration

Initialize the list of wavelength and a list of files corresponding to monochromatic wavelengths flatfields

In [4]:
par.lamlist = lamlist
par.filelist = np.sort(glob.glob(par.wavecalDir + "det*.fits"))


Run the main calibration routine; this creates the wavelength solution, the high-resolution PSFlets, and the polychrome cube which is used in least squares extraction.

In [5]:
from tools.wavecal import buildcalibrations
buildcalibrations(par,parallel=True,genwavelengthsol=False)

2017-02-01 20:01:55,172: INFO     Building calibration files, placing results in /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/wavecal/
2017-02-01 20:01:55,192: INFO     Read data from HDU 1 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/wavecal/det_605.fits
2017-02-01 20:01:55,197: INFO     Read data from HDU 1 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/wavecal/det_605.fits
2017-02-01 20:01:55,209: INFO     Read data from HDU 1 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/wavecal/det_615.fits
2017-02-01 20:01:55,220: INFO     Read data from HDU 1 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/wavecal/det_625.fits
2017-02-01 20:01:55,232: INFO     Read data from HDU 1 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/wavecal/det_635.fits
2017-02-01 20:01:55,243: INFO     Read data from HDU 1 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/wavecal/det_645.fits
2017-02-01 20:01:55,254: INFO     Read data from HDU 1 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/wa

Total time elapsed: 113 s


# Create an artificial flatfield

In [3]:
from unitTests import testCreateFlatfield
testCreateFlatfield(par,nlam=150,parallel=False)

2017-02-01 21:51:19,851: INFO     The number of input pixels per lenslet is 3.000000
2017-02-01 21:51:19,852: INFO     The plate scale of the input cube is 58.000000 um/pixel
2017-02-01 21:51:19,853: INFO     Import all kernels and rescale them to same plate scale
2017-02-01 21:51:19,854: INFO     Loading spot diagrams.
2017-02-01 21:51:20,008: INFO     kernel scale average is 1.706 micron per pixel at 890 nm
2017-02-01 21:51:20,009: INFO     Resampling kernels to match input
2017-02-01 21:51:20,100: INFO     pxprlens: 184.000
2017-02-01 21:51:20,148: INFO     Loading spot diagrams.
2017-02-01 21:51:20,198: INFO     kernel scale average is 1.475 micron per pixel at 770 nm
2017-02-01 21:51:20,199: INFO     Resampling kernels to match input
2017-02-01 21:51:20,263: INFO     Padding smaller kernels
2017-02-01 21:51:20,309: INFO     Loading spot diagrams.
2017-02-01 21:51:20,355: INFO     kernel scale average is 1.264 micron per pixel at 660 nm
2017-02-01 21:51:20,356: INFO     Resampling 

# Reduction

In [4]:
from IFS import reduceIFSMap
fname='/Users/mrizzo/IFS/IFS-Simulator/unitTestsOutputs/flatfield.fits'
#fname=par.wavecalDir+'det_655.fits'
reduceIFSMap(par,fname,method='lstsq',ivar=False)
# reduceIFSMap(par,fname,method='intopt')
# reduceIFSMap(par,fname,method='apphot')

2017-02-01 23:18:24,616: INFO     Read data from HDU 1 of /Users/mrizzo/IFS/IFS-Simulator/unitTestsOutputs/flatfield.fits


# Example

In [3]:
from tools.image import Image
mperpix = 3./4.*par.pitch
par.pixperlenslet = par.pitch/mperpix
par.mperpix = mperpix
wavlist = np.linspace(605,725,31)
ifs_psf_onax_contrast = Image(filename='ifs_psf_onax_contrast.fits').data
from IFS import propagateIFS
finalFrame = propagateIFS(par,wavlist*1e-3,ifs_psf_onax_contrast,name='optfittest',parallel=False)

2017-02-02 00:34:49,071: INFO     Read data from HDU 1 of ifs_psf_onax_contrast.fits
2017-02-02 00:34:51,403: INFO     The number of input pixels per lenslet is 1.333333
2017-02-02 00:34:51,403: INFO     The plate scale of the input cube is 130.500000 um/pixel
2017-02-02 00:34:51,404: INFO     Import all kernels and rescale them to same plate scale
2017-02-02 00:34:51,405: INFO     Loading spot diagrams.
2017-02-02 00:34:51,519: INFO     kernel scale average is 1.706 micron per pixel at 890 nm
2017-02-02 00:34:51,521: INFO     Resampling kernels to match input
2017-02-02 00:34:51,601: INFO     pxprlens: 184.000
2017-02-02 00:34:51,649: INFO     Loading spot diagrams.
2017-02-02 00:34:51,738: INFO     kernel scale average is 1.475 micron per pixel at 770 nm
2017-02-02 00:34:51,739: INFO     Resampling kernels to match input
2017-02-02 00:34:51,804: INFO     Padding smaller kernels
2017-02-02 00:34:51,850: INFO     Loading spot diagrams.
2017-02-02 00:34:51,935: INFO     kernel scale ave

In [4]:
from IFS import reduceIFSMap
fname='/Users/mrizzo/IFS/IFS-Simulator/SimResults/optfittest.fits'
#fname=par.wavecalDir+'det_655.fits'
reduceIFSMap(par,fname,method='intopt',ivar=False)


2017-02-02 00:35:25,449: INFO     Read data from HDU 1 of /Users/mrizzo/IFS/IFS-Simulator/SimResults/optfittest.fits
2017-02-02 00:35:25,470: INFO     Reduced cube will have 21 wavelength bins
tools/reduction.py:548: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  pix_center_vals = [np.sum(im.data[i1:i1 + delt_y, val]*gaussian) for val in ix]/weights
2017-02-02 00:35:55,668: INFO     Writing data to /Users/mrizzo/IFS/IFS-Simulator/SimResults/optfittest_red_intopt.fits


array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       ..., 
       [[ nan,  nan, 